In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [4]:
version = 'v8'
data_path = Path('data/midi')
version_path = data_path/version

In [5]:
import pandas as pd

In [6]:
source_dir = 'midi_sources'
out_dir = 'midi_transform'
source_csv = version_path/'metadata'/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'{out_dir}.csv'
out_csv.parent.mkdir(parents=True, exist_ok=True)
source_csv, out_csv

(PosixPath('data/midi/v8/metadata/midi_sources.csv'),
 PosixPath('data/midi/v8/midi_transform/midi_transform.csv'))

In [7]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,4,15,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,ht_offset,ht_bpm,artist,ht_key,ht_time_signature,bpm,section,parts,genres,mxl
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1.0,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,...,0.0,85.0,wayne-sharpe,C,3.0,85.0,intro,"intro,chorus",NaN,NaN
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,0.0,128.0,wayne-sharpe,C,4.0,128.0,chorus,"intro,chorus",NaN,NaN
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,0.0,96.0,what-a-day,C,4.0,96.0,chorus,chorus,Jazz,NaN
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1.0,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,0.0,128.0,weezer,C,4.0,NaN,intro-and-verse,intro-and-verse,NaN,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,0.0,93.0,weezer,C,4.0,93.0,chorus,chorus,Rock,NaN


In [8]:
df_filtered = df.loc[df.time_signature.notna()]

In [9]:
from data_sources import process_parallel, transform_csv_row

In [10]:
from midi_transform import *

### Need better midi formatting

In [11]:
def transform_func(file, out_file, row):
    offset = row.ht_offset if row.source == 'hooktheory' else row.inferred_offset
    music_file = compress_midi_file(file, cutoff=5) # remove non note tracks and standardize instruments
    music_file.open(out_file, attrib='wb')
    music_file.write()
    music_file.close()

In [12]:
from functools import partial
parallel_func = partial(transform_csv_row, 
        transform_func=transform_func,
        base_path=version_path,
        source_dir=source_dir,
        out_dir=out_dir,
        out_extension=None,
        source_col='midi'
       )

In [13]:
# sanity check
# for r in df_filtered.iterrows():
#     parallel_func(r)

In [14]:
### AS TOD: Fix 'info channel is not channel 0' error

In [15]:
idx2out = process_parallel(parallel_func, df_filtered.iterrows(), total=df_filtered.shape[0])

Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-dance-eletric/Taylor Dayne - Love Will Lead You Back.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-dance-eletric/Taylor Dayne - Send Me A Lover.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-dance-eletric/Whitney Houston - Heartbreak Hotel (feat. faith evens & kelly price).mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-dance-eletric/ATB - Killer.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-dance-eletric/Madonna - Get Into The Groove.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-dance-eletric/Whitney Hous

Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Celine Dion - Beauty And The Beast.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Peter Gabriel - Starless.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Tom Jones - Sexbomb.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Cyndi Lauper - I Drove All Night.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Bruno e Marrone - Um bom Perdedor.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Whitney Houston - Heartbreak Hotel (feat. faith evens & kelly price).mid
Error converting midi to sequence 

Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Daniel Bedingfield - Friday.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Puff Daddy - Bad Boys For Life.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Whitney Houston - All The Man I Need.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Bruno e Marrone - Preciso Amar de Novo.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Black Eyed Peas - Request Line.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Carpenters - Don't Cry For Me Argentina.mid
Error converting midi to sequence 'NoneType'

Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Carpenters - We've Only Just Begun.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Sting - Soul Cages.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Madonna - American Life.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Ricky Nelson - Hello Mary Lou.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/ABBA - Dancing Queen.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/freemidi/genre-pop/Ace of Base - Cest la vie always.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8

Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/midiworld/named_midi/The_Searchers_-_Needles_and_Pins.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/midiworld/named_midi/Sugar_Ray_-_Every_Morning.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/midiworld/named_midi/Prince_-_When_Doves_Cry.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/midiworld/named_midi/Jackson_Browne_-_Somebodys_Baby.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/midiworld/named_midi/Michael_Jackson_-_Rock_with_You.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/midiworld/named_midi/Vanilla_Ice_-_Ice_Ice_Baby.mid
Error converting midi to sequence 'NoneType' object has no attri

...

midi: WARNING: Conversion error for <MidiEvent None, t=None, track=0, channel=None>: unknown midi event type: None; ignored.


Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/2/218.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/2/2story.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/2/2_pac-until_the_end_of_time.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/R/R/R_Kelly_-_Half_On_A_Baby.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/T/T/TWISTMIX.MID
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/R/R/ryo.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/2/21st_century_schizoid_man.mid
Error converting midi to sequence 'NoneType' object

midi: WARNING: Conversion error for <MidiEvent None, t=None, track=11, channel=None>: unknown midi event type: None; ignored.


Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/R/R/Radio-Ga-Ga-3.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/R/R/Rocket man.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/R/R/Robert_John_-_The_Lion_Sleeps_Tonight.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/R/R/rowboat2.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/R/R/RTL-TUNE.MID
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/R/R/Romance-KV466-2.mid
Error converting midi to sequence 'NoneType' object has no attribute 'open' data/midi/v8/midi_sources/130k_reddit/R/R/rem-immitation_of_life.mid
Error converting midi to s

midi: WARNING: Conversion error for <MidiEvent None, t=None, track=0, channel=None>: unknown midi event type: None; ignored.


In [16]:
tdf = pd.DataFrame(data={out_dir: list(idx2out.values())}, index=list(idx2out.keys()))

In [17]:
merged_df = df.join(tdf)

In [18]:
tdf.shape, df.shape, merged_df.shape

((166644, 1), (185846, 22), (185846, 23))

In [20]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,ht_bpm,artist,ht_key,ht_time_signature,bpm,section,parts,genres,mxl,midi_transform
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1.0,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,...,85.0,wayne-sharpe,C,3.0,85.0,intro,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,128.0,wayne-sharpe,C,4.0,128.0,chorus,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,96.0,what-a-day,C,4.0,96.0,chorus,chorus,Jazz,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1.0,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,128.0,weezer,C,4.0,NaN,intro-and-verse,intro-and-verse,NaN,NaN,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,93.0,weezer,C,4.0,93.0,chorus,chorus,Rock,NaN,midi_transform/hooktheory/pianoroll/w/weezer/f...
